- Category, url, title - missing in conflictQA. Category (for the author experiment) can be added with additional prompting of LLM?
- Stance, by definition, has been assigned to be the same as search_type, but cross-check this?
- Dropped popularity, ground_truth, and the shorter parametric and counter answers from conflictQA.
- Remove quotes from the search_engine_input in conflictingQA for consistency?
- Note: text_window is currently not necessarily 512 tokens in conflictQA?

To Do:
- Use an entailment model to check if search_engine_input matches up with parametric or counter memory evidence and then assign to the yes or no search_type and stance?

In [81]:
import pandas as pd
import json

with open('../conflictQA/conflictQA-strategyQA-chatgpt.json', 'r') as file:
    data = [json.loads(line) for line in file]

df = pd.DataFrame(data)

In [82]:
# Set display options to show all columns
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)

# Print the DataFrame in a scrollable manner (without wrapping and truncation)
print(df)

                                               question  popularity ground_truth                                      memory_answer                                  parametric_memory                                     counter_answer                                     counter_memory                 parametric_memory_aligned_evidence                    counter_memory_aligned_evidence
0     Are more people today related to Genghis Khan ...      273182       [True]  More people today are related to Genghis Khan ...  It is believed that more people today are rela...  More people today are not related to Genghis K...  Recent studies have shown that the genetic leg...  Julius Caesar had three children. Genghis Khan...  According to a recent study conducted by the N...
1     Could the members of The Police perform lawful...      112137      [False]  The members of The Police could not perform la...  The members of The Police, a British rock band...  The members of The Police can perform lawf

In [83]:
def process_dataframe(df):
    df = df.rename(columns={'question': 'search_query'})

    # Duplicate the rows
    df_yes = df.copy()
    df_no = df.copy()

    # Take each row, duplicate it. For the first duplicate, rename parametric_memory_aligned_evidence to text_window, and add a column search_type with the value yes_statement. For the second duplicate, rename counter_memory_aligned_evidence to text_window, and add a column search_type with the value no_statement. Drop both the parametric and counter column after that.

    # For the first duplicate
    df_yes = df_yes.rename(columns={'parametric_memory_aligned_evidence': 'text_window'})
    df_yes['search_type'] = 'yes_statement'

    # For the second duplicate
    df_no = df_no.rename(columns={'counter_memory_aligned_evidence': 'text_window'})
    df_no['search_type'] = 'no_statement'

    # Initialize an empty list to collect interleaved rows
    interleaved_rows = []

    # Interleaving rows alternatively
    for row_yes, row_no in zip(df_yes.iterrows(), df_no.iterrows()):
        interleaved_rows.append(row_yes[1])  # row_yes[1] gives the actual row as a Series
        interleaved_rows.append(row_no[1])   # row_no[1] gives the actual row as a Series

    # Create a new DataFrame from the interleaved rows
    result = pd.DataFrame(interleaved_rows).reset_index(drop=True)

    # Drop the original columns
    result = result.drop(columns=['parametric_memory_aligned_evidence', 'counter_memory_aligned_evidence', 'parametric_memory', 'memory_answer', 'counter_answer', 'counter_memory', 'ground_truth', 'popularity'])

    return result

# Process the DataFrame
result_df = process_dataframe(df)

# Display the resulting DataFrame
print(result_df)

                                           search_query                                        text_window    search_type
0     Are more people today related to Genghis Khan ...  Julius Caesar had three children. Genghis Khan...  yes_statement
1     Are more people today related to Genghis Khan ...  According to a recent study conducted by the N...   no_statement
2     Could the members of The Police perform lawful...  The members of The Police were musicians, not ...  yes_statement
3     Could the members of The Police perform lawful...  The Police is a well-respected band that has s...   no_statement
4         Would a dog respond to bell before Grey seal?  Grey seals have no ear flaps and their ears ca...  yes_statement
...                                                 ...                                                ...            ...
2485  Are selfies more dangerous than plague in mode...  There are an average of 7 human plague cases r...   no_statement
2486  Is a cory catfish 

In [84]:
result_df.to_csv('pre_search_engine_input.csv', index=False)

In [85]:
!python query_generator.py

/Users/nikitamenon/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (
More people today are related to Genghis Khan than Julius Caesar.
More people today are not related to Genghis Khan than to Julius Caesar.
The members of The Police could perform lawful arrests.
The members of The Police could not perform lawful arrests.


In [88]:
df = pd.read_csv('post_search_engine_input.csv')

# Function to assign stance based on search_type
def assign_stance(row):
    if row['search_type'] == 'yes_statement':
        return 'yes'
    elif row['search_type'] == 'no_statement':
        return 'no'
    else:
        return None

# Apply the function to create the new column 'stance'
df['stance'] = df.apply(assign_stance, axis=1)

# Add new empty column 'category'
df['category'] = None

# Rearrange the columns
desired_order = ['category', 'search_query', 'search_type', 'search_engine_input', 'text_window', 'stance']
df = df[desired_order]

print(df)

  category                                       search_query    search_type                                search_engine_input                                        text_window stance
0     None  Are more people today related to Genghis Khan ...  yes_statement  More people today are related to Genghis Khan ...  Julius Caesar had three children. Genghis Khan...    yes
1     None  Are more people today related to Genghis Khan ...   no_statement  More people today are not related to Genghis K...  According to a recent study conducted by the N...     no
2     None  Could the members of The Police perform lawful...  yes_statement  The members of The Police could perform lawful...  The members of The Police were musicians, not ...    yes
3     None  Could the members of The Police perform lawful...   no_statement  The members of The Police could not perform la...  The Police is a well-respected band that has s...     no


In [91]:
df.to_csv('altered_format_conflictQA.csv', index=False)